In [1]:
import pandas as pd
import os
import re

os.chdir(r'C:\Users\jf79\Repos\REAP-Analytics')
import myfuncs.myfuncs as mf

mf.read_directory(r"C:\Users\jf79\Downloads")

import warnings
warnings.filterwarnings('ignore')

Directory being read is: C:\Users\jf79\Downloads
DataFrame: files
Number of records: 96 | Number of fields: 1
Number of unique duplicate records: 0 | Total number of duplicate records: 0


,File Names
0,145KS Visitor Enquiry Form(1-5000) (1).xlsx
1,145KS Visitor Enquiry Form(1-5000) (2).xlsx
2,145KS Visitor Enquiry Form(1-5000).xlsx
3,145KS Visitor Enquiry Form(10001-12548).xlsx
4,145KS Visitor Enquiry Form(10001-12745).xlsx
...,...
91,UnitRegisterBlankByFlight (2).xlsx
92,UnitRegisterBlankByFlight.xlsx
93,workforce - Green Book May 2024 - Accessible ...
94,Workspace IDs.xlsx


In [ ]:
granicus_case_data = mf.query_data(schema='granicus', data='GranicusRedshift_Cases')
granicus_process_data = mf.query_data(schema='granicus', data='GranicusRedshift_Processes')

In [ ]:
from pandas.tseries.offsets import MonthBegin, DateOffset
import calendar

latest_date = pd.to_datetime(granicus_case_data['date_completed']).max()
first_of_current_month = latest_date.replace(day=1)
last_full_month = first_of_current_month - MonthBegin(1)
same_month_last_year = last_full_month - DateOffset(years=1)
last_full_month = last_full_month.strftime('%Y-%m')
same_month_last_year = same_month_last_year.strftime('%Y-%m')
last_full_month_name = calendar.month_abbr[latest_date.month-1]
month_year = f'{last_full_month_name}-{(first_of_current_month - MonthBegin(1)).year}'

In [ ]:
mf.validate_data(granicus_case_data)

#####################################################################################################################################################################
Validation Dataframe
Number of records: 908,587 | Number of fields: 11
Number of unique duplicate records: 0 | Total number of duplicate records: 0


,case_id,case_status,process_id,date_started,originator,view_group_id,date_due,date_due_warn,date_completed,is_anonymized,citizen_id
0,FS-Case-415740763,closed,AF-Process-4a120a3d-e18e-4e1f-978c-e72d8e522d32,2022-04-15 20:00:48,Corrkatie@gmail.com,USERGROUP-57669b64-8c88-45b4-8c48-768cee308480,NaT,None,2022-04-15 20:00:48,0,712352674
1,FS-Case-415743040,closed,AF-Process-8ebd2441-f685-4c67-a8a6-7ddc4c7a486f,2022-04-15 20:06:21,wendykhajenouri@icloud.com,USERGROUP-a0229ff0-0cd0-472b-8f1d-814d3497ce58,NaT,None,2022-04-15 20:06:21,0,742699509
2,FS-Case-415746107,closed,AF-Process-8ebd2441-f685-4c67-a8a6-7ddc4c7a486f,2022-04-15 20:03:42,samsultan2007@yahoo.co.uk,USERGROUP-a0229ff0-0cd0-472b-8f1d-814d3497ce58,NaT,None,2022-04-15 20:03:43,0,78649422
3,FS-Case-415747162,closed,AF-Process-51b31e20-396c-458e-8119-1ac6cc1ab704,2022-04-15 20:21:11,0,USERGROUP-7050c12e-0169-4c8b-be7d-7281db46e156,NaT,None,2022-04-19 10:16:59,0,
4,FS-Case-415752262,closed,AF-Process-6908c150-7bc5-49ba-84ff-1e1b78403ef8,2022-04-15 21:48:02,rickaelmodeste@hotmail.co.uk,,NaT,None,2022-04-15 21:48:03,0,7843104
...,...,...,...,...,...,...,...,...,...,...,...
908582,LINK507498182,closed,AF-Process-8ebd2441-f685-4c67-a8a6-7ddc4c7a486f,2023-04-13 13:40:52,lanamwalther@gmail.com,USERGROUP-a0229ff0-0cd0-472b-8f1d-814d3497ce58,NaT,None,2023-04-13 13:40:53,0,789722442
908583,LINK507509424,closed,AF-Process-8ebd2441-f685-4c67-a8a6-7ddc4c7a486f,2023-04-13 14:05:37,r.g.mcf@btinternet.com,USERGROUP-a0229ff0-0cd0-472b-8f1d-814d3497ce58,NaT,None,2023-04-13 14:05:38,0,764776090
908584,LINK507524552,closed,AF-Process-8ebd2441-f685-4c67-a8a6-7ddc4c7a486f,2023-04-13 14:44:30,clairedrewett23@gmail.com,USERGROUP-a0229ff0-0cd0-472b-8f1d-814d3497ce58,NaT,None,2023-04-13 14:44:30,0,791204179
908585,LINK507560501,closed,AF-Process-8ebd2441-f685-4c67-a8a6-7ddc4c7a486f,2023-04-13 16:05:04,soledelgadillo@gmail.com,USERGROUP-a0229ff0-0cd0-472b-8f1d-814d3497ce58,NaT,None,2023-04-13 16:05:05,0,739618586


Unique Values
DataFrame: unique_df


,case_id,case_status,process_id,date_started,originator,view_group_id,date_due,date_due_warn,date_completed,is_anonymized,citizen_id
0,ASC687018248,closed,AF-Process-0093bd07-187e-43fb-85af-209efd0489e0,2019-04-17 08:30:29,0,,2020-08-08 16:55:50,--,2019-04-17 08:30:30,0,
1,ASC687027664,open,AF-Process-00fe9fe8-1434-4a71-8d29-82faa6b250ff,2019-04-17 08:54:20,00139990RichaCOP@IM-COP.com,USERGROUP-00caf923-5e90-4fba-a8f2-6726198fd348,2020-08-08 17:13:40,--,2019-04-17 08:54:20,1,0
2,ASC687064680,--,AF-Process-018a989a-2c24-4cc7-866d-2e039a101c11,2019-04-17 08:55:33,002sbush@amigosburgersandshakes.com,USERGROUP-070ca00c-bc47-405c-b5d8-15625618813d,2020-08-10 15:16:20,--,2019-04-17 08:55:33,--,710000313
3,ASC687205670,--,AF-Process-03344195-0269-485f-bc87-771f020dabaf,2019-04-17 09:05:34,00karimwork@gmail.com,USERGROUP-095bb8a3-6144-45d3-b458-42099475c03d,2020-08-16 13:29:18,--,2019-04-17 09:05:35,--,710001692
4,ASC687251513,--,AF-Process-03952aeb-b520-4f54-a1bd-709e5c9e6e97,2019-04-17 09:11:53,00realiz@gmail.com,USERGROUP-0d77fb26-b838-48dd-9fb6-1dca0ace6ac4,2020-08-20 12:11:16,--,2019-04-17 09:11:53,--,710002574
5,ASC687357754,--,AF-Process-055c19ab-4906-4f62-acd6-c9da7e2ce2eb,2019-04-17 09:16:03,01082003rawan@gmail.com,USERGROUP-1239c20c-5d3f-4229-a3b0-37f5fe11dd18,2020-08-20 14:36:07,--,2019-04-17 09:16:04,--,71000302
6,ASC687372692,--,AF-Process-05c5d38a-3a36-4fe8-b384-2d816f5758fb,2019-04-17 09:18:10,010fulham@amigosburgersandshakes.com,USERGROUP-131ef799-7f63-413a-afd8-5d6de7a6f0ef,2020-08-20 18:35:21,--,2019-04-17 09:18:11,--,710003471
7,ASC687377747,--,AF-Process-066c1469-1e43-4b31-bcfe-2c50f36ee032,2019-04-17 09:19:49,01harper.sfb@googlemail.com,USERGROUP-14cff18c-02cb-4634-9b44-78a343ffe6f5,2020-08-20 19:17:35,--,2019-04-17 09:19:50,--,71000363
8,ASC687379618,--,AF-Process-070e18a7-351b-4140-bba7-fb7e74c50c1e,2019-04-17 09:26:00,01tdavies@gmail.com,USERGROUP-1c957769-fa64-41b9-be8b-08da0afcedca,2020-08-22 15:06:47,--,2019-04-17 09:26:01,--,710003741
9,ASC687383067,--,AF-Process-0c4d4293-aebd-46ef-8053-8a120e5bda22,2019-04-17 09:30:10,0222mjmj@gmail.com,USERGROUP-27eea669-6f9f-4087-92b7-4ce83a266c51,2020-08-26 00:00:00,--,2019-04-17 09:30:10,--,710003848



DataFrame: info_df
Max number of non-null records: 908,587


,Field Name,No. of Unique Values,No. of Null Values,No. of Blank Values,Data Type
0,case_id,"908,587",--,--,object
1,case_status,2,--,--,object
2,process_id,235,--,--,object
3,date_started,"904,063",--,--,datetime64[ns]
4,originator,"148,763",--,--,object
5,view_group_id,79,--,"97,515",object
6,date_due,777,"907,523",--,datetime64[ns]
7,date_due_warn,0,Empty,--,object
8,date_completed,"875,636","27,167",--,datetime64[ns]
9,is_anonymized,2,--,--,object



Summary statistics:


,date_started,date_due,date_completed
count,908587,1064,881420
mean,2022-11-27 06:54:33.757782528,2022-12-18 09:01:23.256578048,2022-11-16 15:50:52.330721792
min,2019-04-17 08:30:29,2020-08-08 16:55:50,2019-04-17 08:30:30
25%,2021-09-21 10:20:06.500000,2021-08-04 06:00:00,2021-09-05 19:24:47
50%,2022-12-21 15:18:20,2022-11-18 00:00:00,2022-12-02 08:21:08
75%,2024-03-15 10:09:03.500000,2024-04-22 06:00:00,2024-03-04 08:52:58.750000128
max,2025-08-29 01:44:52,2025-09-04 00:00:00,2025-08-29 01:44:57


End of data validation
#####################################################################################################################################################################


In [ ]:
columns_to_drop = [
    'date_due_warn','date_due','citizen_id'
]
granicus_case_data.drop(columns=columns_to_drop, inplace=True)

In [ ]:
path_to_data = r"C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Granicus"
mf.read_directory(directory=path_to_data)
os.chdir(path_to_data)
lookup_data = pd.read_excel("Granicus Process List.xlsx")

Directory being read is: C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Granicus
DataFrame: files
Number of records: 2 | Number of fields: 1
Number of unique duplicate records: 0 | Total number of duplicate records: 0


,File Names
0,Granicus E-Process Report.xlsx
1,Granicus Process List.xlsx


In [ ]:
report_data = pd.merge(
    granicus_case_data, granicus_process_data,
    how='left', on='process_id'
)

report_data['month_of_form'] = pd.to_datetime(report_data['date_completed']).dt.strftime("%Y-%m")

report_data = report_data.groupby(['month_of_form', 'process_name','process_id'])['case_id'].count().reset_index().sort_values(by=['month_of_form', 'case_id'], ascending=False)
report_data = report_data[report_data['month_of_form'] == last_full_month]

report_data = pd.merge(
    report_data, lookup_data,
    how='left', on='process_id',
    suffixes=['','_drop']
)

report_data = report_data[report_data['process_name_drop'].isna() == False]
report_data.rename(
    columns={
        'process_name':'E-Process',
        'case_id':month_year
    },
    inplace=True
)
report_data.drop(
    columns=['month_of_form','process_id','process_name_drop'],
    inplace=True
)

report_data[r'% Cummulative Total'] = report_data[month_year].cumsum() / report_data[month_year].sum()

report_data = report_data[report_data[r'% Cummulative Total'] <= 0.9]
mf.display(report_data)

mf.export_file(
    report_data,
    directory = path_to_data,
    df_name = 'Granicus E-Process Report',
    file_type='xlsx'
)

DataFrame: report_data
Number of records: 25 | Number of fields: 3
Number of unique duplicate records: 0 | Total number of duplicate records: 0


,E-Process,Jul-2025,% Cummulative Total
1,Council tax moving in moving out,2084,0.235294
2,Contact parking permits,1285,0.380377
3,Housing advice,709,0.460427
4,Bulky waste collection,607,0.528960
5,Apply for a parking suspension,579,0.594332
6,"Smart sacks, order top-up",342,0.632946
7,Housing repairs reporting form,316,0.668624
9,In-year admissions application,219,0.693350
10,Report a problem,205,0.716495
11,Apply to get advice to prevent homelessness,159,0.734447


Exporting Granicus E-Process Report to XLSX...
@ C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Granicus\Granicus E-Process Report.xlsx

Error exporting to XLSX: can't set attribute
